In [1]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext(appName="Examen")

22/05/13 09:13:04 WARN Utils: Your hostname, DESKTOP-O781O4L resolves to a loopback address: 127.0.1.1; using 192.168.56.1 instead (on interface eth1)
22/05/13 09:13:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/13 09:13:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
lines = sc.textFile("appl.log")

In [3]:
from pyspark.sql.session import SparkSession
spark = SparkSession(sc)

df = spark.read.csv("housing.csv")

df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("housing.csv")

file = spark.read \
    .option("header", "true") \#Si hay encabezado lo carga (nombres columnas)
    .option("inferSchema", "true") \#Spark da forma a los datos (si no todo serán strings) 
    .option("nullValue", "NA") \#Null values se cambia a NA
    .option("mode", "dropMalformed") \#Fuera filas malformadas (distintas)
    .csv('exercise_files/favourite_animals.csv')

SyntaxError: unexpected character after line continuation character (2180340017.py, line 12)

In [7]:
sc.stop()

In [8]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext(appName="SparkDFBasics")
from pyspark.sql.session import SparkSession
spark = SparkSession(sc)

In [25]:
file = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("nullValue", "NA") \
    .option("mode", "dropMalformed") \
    .csv('Semana9/airports.csv')

In [26]:
file

DataFrame[IATA_CODE: string, AIRPORT: string, CITY: string, STATE: string, COUNTRY: string, LATITUDE: double, LONGITUDE: double]

In [27]:
#shape of dataset
print((file.count(),len(file.columns)))

(322, 7)


In [28]:
file.show()

+---------+--------------------+-------------+-----+-------+--------+----------+
|IATA_CODE|             AIRPORT|         CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|
+---------+--------------------+-------------+-----+-------+--------+----------+
|      ABE|                null|    Allentown|   PA|    USA|40.65236|  -75.4404|
|      ABI|Abilene Regional ...|      Abilene|   TX|    USA|    null|  -99.6819|
|      ABQ|Albuquerque Inter...|  Albuquerque|   NM|    USA|35.04022|-106.60919|
|      ABR|Aberdeen Regional...|     Aberdeen|   SD|    USA|45.44906| -98.42183|
|      ABY|Southwest Georgia...|       Albany|   GA|    USA|31.53552| -84.19447|
|      ACK|Nantucket Memoria...|    Nantucket|   MA|    USA|41.25305| -70.06018|
|      ACT|Waco Regional Air...|         Waco|   TX|    USA|31.61129| -97.23052|
|      ACV|      Arcata Airport|Arcata/Eureka|   CA|    USA|40.97812|-124.10862|
|      ACY|Atlantic City Int...|Atlantic City|   NJ|    USA|39.45758| -74.57717|
|      ADK|        Adak Airp

In [29]:
file.registerTempTable("airports")

/opt/spark/python/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


In [30]:
df_subset=spark.sql("""
select
 IATA_CODE,
 AIRPORT,
 CITY,
 STATE,
 COUNTRY,
 LATITUDE,
 LONGITUDE
FROM
 airports
"""
)

In [31]:
df_subset.head(5)

[Row(IATA_CODE='ABE', AIRPORT=None, CITY='Allentown', STATE='PA', COUNTRY='USA', LATITUDE=40.65236, LONGITUDE=-75.4404),
 Row(IATA_CODE='ABI', AIRPORT='Abilene Regional Airport', CITY='Abilene', STATE='TX', COUNTRY='USA', LATITUDE=None, LONGITUDE=-99.6819),
 Row(IATA_CODE='ABQ', AIRPORT='Albuquerque International Sunport', CITY='Albuquerque', STATE='NM', COUNTRY='USA', LATITUDE=35.04022, LONGITUDE=-106.60919),
 Row(IATA_CODE='ABR', AIRPORT='Aberdeen Regional Airport', CITY='Aberdeen', STATE='SD', COUNTRY='USA', LATITUDE=45.44906, LONGITUDE=-98.42183),
 Row(IATA_CODE='ABY', AIRPORT='Southwest Georgia Regional Airport', CITY='Albany', STATE='GA', COUNTRY='USA', LATITUDE=31.53552, LONGITUDE=-84.19447)]

In [35]:
def null_string(element):
    if (element == None):
        return ""
    else:
        return element
    
spark.udf.register("null_string", lambda x : null_string(x))

def null_num(element):
    if (element == None):
        return 0
    else:
        return element
    
spark.udf.register("null_num", lambda x : null_num(x))

22/05/13 09:33:11 WARN SimpleFunctionRegistry: The function null_string replaced a previously registered function.
22/05/13 09:33:11 WARN SimpleFunctionRegistry: The function null_num replaced a previously registered function.


<function __main__.<lambda>(x)>

In [36]:
query = """
SELECT
    IATA_CODE,
    null_string(AIRPORT) as AIRPORT,
    CITY,
    STATE,
    COUNTRY,
    null_num(LATITUDE) as LATITUDE,
    LONGITUDE
FROM 
    airports 
"""
datos = spark.sql(query).coalesce(1) \
    .write \
    .option("header","true") \
    .option("sep",",") \
    .mode("overwrite") \
    .csv("airport_limpio.csv")

In [37]:
file = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("nullValue", "NA") \
    .option("mode", "dropMalformed") \
    .csv('airport_limpio.csv')

file.show()

+---------+--------------------+-------------+-----+-------+--------+----------+
|IATA_CODE|             AIRPORT|         CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|
+---------+--------------------+-------------+-----+-------+--------+----------+
|      ABE|                    |    Allentown|   PA|    USA|40.65236|  -75.4404|
|      ABI|Abilene Regional ...|      Abilene|   TX|    USA|     0.0|  -99.6819|
|      ABQ|Albuquerque Inter...|  Albuquerque|   NM|    USA|35.04022|-106.60919|
|      ABR|Aberdeen Regional...|     Aberdeen|   SD|    USA|45.44906| -98.42183|
|      ABY|Southwest Georgia...|       Albany|   GA|    USA|31.53552| -84.19447|
|      ACK|Nantucket Memoria...|    Nantucket|   MA|    USA|41.25305| -70.06018|
|      ACT|Waco Regional Air...|         Waco|   TX|    USA|31.61129| -97.23052|
|      ACV|      Arcata Airport|Arcata/Eureka|   CA|    USA|40.97812|-124.10862|
|      ACY|Atlantic City Int...|Atlantic City|   NJ|    USA|39.45758| -74.57717|
|      ADK|        Adak Airp

In [13]:
#Predecir status
df=spark.read.csv('Semana12/Log_Reg_dataset.csv',inferSchema=True,header=True)
df.describe().show()

+-------+--------+-----------------+-----------------+--------+-----------------+------------------+
|summary| Country|              Age|   Repeat_Visitor|Platform| Web_pages_viewed|            Status|
+-------+--------+-----------------+-----------------+--------+-----------------+------------------+
|  count|   20000|            20000|            20000|   20000|            20000|             20000|
|   mean|    null|         28.53955|           0.5029|    null|           9.5533|               0.5|
| stddev|    null|7.888912950773227|0.500004090187782|    null|6.073903499824976|0.5000125004687693|
|    min|  Brazil|               17|                0|    Bing|                1|                 0|
|    max|Malaysia|              111|                1|   Yahoo|               29|                 1|
+-------+--------+-----------------+-----------------+--------+-----------------+------------------+



In [14]:
#converting categorical data to numerical form  (Country y Platform)
#import required libraries
from pyspark.ml.feature import StringIndexer

In [15]:
#Indexing
search_engine_class = StringIndexer(inputCol="Platform", outputCol="Platform_Num")
search_engine_obj = search_engine_class.fit(df)
df = search_engine_obj.transform(df)

In [16]:
from pyspark.ml.feature import OneHotEncoder
#one hot encoding
search_engine_encoder = OneHotEncoder(inputCol="Platform_Num", outputCol="Platform_Vector")
ohe = search_engine_encoder.fit(df)
df = ohe.transform(df)

In [17]:
country_indexer = StringIndexer(inputCol="Country", outputCol="Country_Num").fit(df)
df = country_indexer.transform(df)

country_encoder = OneHotEncoder(inputCol="Country_Num", outputCol="Country_Vector")
ohe = country_encoder.fit(df)
df = ohe.transform(df)

In [18]:
from pyspark.ml.feature import VectorAssembler
df_assembler = VectorAssembler(inputCols=['Platform_Vector','Country_Vector','Age', 'Repeat_Visitor','Web_pages_viewed'], outputCol="features")
df = df_assembler.transform(df)

In [19]:
#select data for building model
model_df=df.select(['features','Status'])

from pyspark.ml.classification import LogisticRegression

In [20]:
#split the data 
training_df,test_df=model_df.randomSplit([0.75,0.25])

In [21]:
log_reg_class=LogisticRegression(labelCol='Status')
log_reg_model=log_reg_class.fit(training_df)

In [22]:
#Predicción del train
train_results=log_reg_model.evaluate(training_df).predictions

/opt/spark/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [24]:
train_results.show()

+--------------------+------+--------------------+--------------------+----------+
|            features|Status|       rawPrediction|         probability|prediction|
+--------------------+------+--------------------+--------------------+----------+
|(8,[0,2,5,7],[1.0...|     0|[6.00562598362045...|[0.99754121461469...|       0.0|
|(8,[0,2,5,7],[1.0...|     0|[6.00562598362045...|[0.99754121461469...|       0.0|
|(8,[0,2,5,7],[1.0...|     0|[6.00562598362045...|[0.99754121461469...|       0.0|
|(8,[0,2,5,7],[1.0...|     0|[6.00562598362045...|[0.99754121461469...|       0.0|
|(8,[0,2,5,7],[1.0...|     0|[5.24938125199908...|[0.99477666024094...|       0.0|
|(8,[0,2,5,7],[1.0...|     0|[5.24938125199908...|[0.99477666024094...|       0.0|
|(8,[0,2,5,7],[1.0...|     0|[5.24938125199908...|[0.99477666024094...|       0.0|
|(8,[0,2,5,7],[1.0...|     0|[5.24938125199908...|[0.99477666024094...|       0.0|
|(8,[0,2,5,7],[1.0...|     0|[5.24938125199908...|[0.99477666024094...|       0.0|
|(8,

In [46]:
correct_preds=train_results.filter(train_results['Status']==1).filter(train_results['prediction']==1).count()

In [49]:
#accuracy on training dataset 
float(correct_preds)/(training_df.filter(training_df['Status']==1).count())

0.9387428266381956

In [50]:
#Test Set results
results=log_reg_model.evaluate(test_df).predictions

In [51]:
(results.filter(results['Status']==1).filter(results['prediction']==1).count())/(training_df.filter(training_df['Status']==1).count())

0.31295876151074337

In [52]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
#confusion matrix
true_postives = results[(results.Status == 1) & (results.prediction == 1)].count()
true_negatives = results[(results.Status == 0) & (results.prediction == 0)].count()
false_positives = results[(results.Status == 0) & (results.prediction == 1)].count()
false_negatives = results[(results.Status == 1) & (results.prediction == 0)].count()

In [53]:
print (true_postives)
print (true_negatives)
print (false_positives)
print (false_negatives)
print(true_postives+true_negatives+false_positives+false_negatives)
print (results.count())

2345
2347
146
162
5000
5000


In [54]:
recall = float(true_postives)/(true_postives + false_negatives)
print(recall)

0.9353809333865177


In [55]:
precision = float(true_postives) / (true_postives + false_positives)
print(precision)

0.9413890004014452


In [56]:
accuracy=float((true_postives+true_negatives) /(results.count()))
print(accuracy)

0.9384


In [58]:
file = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("nullValue", "NA") \
    .option("mode", "dropMalformed") \
    .csv('Examen2/311_Service_Requests_from_2010_to_Present_5perc.csv')

file.columns


['_c0',
 'Unique Key',
 'Created Date',
 'Closed Date',
 'Agency',
 'Agency Name',
 'Complaint Type',
 'Descriptor',
 'Location Type',
 'Incident Zip',
 'Incident Address',
 'Street Name',
 'Cross Street 1',
 'Cross Street 2',
 'Intersection Street 1',
 'Intersection Street 2',
 'Address Type',
 'City',
 'Landmark',
 'Facility Type',
 'Status',
 'Due Date',
 'Resolution Description',
 'Resolution Action Updated Date',
 'Community Board',
 'BBL',
 'Borough',
 'X Coordinate (State Plane)',
 'Y Coordinate (State Plane)',
 'Open Data Channel Type',
 'Park Facility Name',
 'Park Borough',
 'Vehicle Type',
 'Taxi Company Borough',
 'Taxi Pick Up Location',
 'Bridge Highway Name',
 'Bridge Highway Direction',
 'Road Ramp',
 'Bridge Highway Segment',
 'Latitude',
 'Longitude',
 'Location']